In [1]:
#
# This is a sample Notebook to demonstrate how to read "MNIST Dataset"
#
import numpy as np # linear algebra
import struct
from array import array
from os.path  import join

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)        

In [2]:
#
# Verify Reading Dataset via MnistDataloader class
#
%matplotlib inline
import random
import matplotlib.pyplot as plt

#
# Set file paths based on added MNIST Datasets
#
input_path = './data'
training_images_filepath = join(input_path, 'train-images.idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels.idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images.idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels.idx1-ubyte')

#
# Helper function to show a list of images with their relating titles
#
def show_images(images, title_texts):
    cols = 5
    rows = int(len(images)/cols) + 1
    plt.figure(figsize=(30,20))
    index = 1    
    for x in zip(images, title_texts):        
        image = x[0]        
        title_text = x[1]
        plt.subplot(rows, cols, index)        
        plt.imshow(image, cmap=plt.cm.gray)
        if (title_text != ''):
            plt.title(title_text, fontsize = 15);        
        index += 1

#
# Load MINST dataset
#
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

#
# Show some random training and test images 
#
# images_2_show = []
# titles_2_show = []
# for i in range(0, 10):
#     r = random.randint(1, 60000)
#     images_2_show.append(x_train[r])
#     titles_2_show.append('training image [' + str(r) + '] = ' + str(y_train[r]))    

# for i in range(0, 5):
#     r = random.randint(1, 10000)
#     images_2_show.append(x_test[r])        
#     titles_2_show.append('test image [' + str(r) + '] = ' + str(y_test[r]))    

# show_images(images_2_show, titles_2_show)

In [3]:
import pickle
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
x_train, x_validate, y_train, y_validate = train_test_split(
    x_train, y_train, test_size=(1/6), random_state=42
)

print(len(x_train))      # Output: 50000    5/6th of training data 
print(len(x_validate))   # Output: 10000    1/6th of training data

50000
10000


In [40]:
class NeuralNetwork:
    def __init__(self, input_dim, hidden_layers, output_dim, initialization="xavier", activation="sigmoid"):
        """
        Initialize a neural network with the given dimensions and initialization method.

        Parameters:
            input_dim (int): Number of input features.
            hidden_layers (list of int): List containing the number of units in each hidden layer.
            output_dim (int): Number of output units.
            initialization (str): Initialization method ('xavier', 'he', 'lecun', 'uniform_xavier', 'uniform_he', or 'uniform_lecun').
        """
        self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        self.output_dim = output_dim
        self.initialization = initialization
        self.activation = activation

        # Initialize weights and biases
        self.weights = []
        self.biases = []

        # Determine initialization scaling factor
        def init_weight(shape, fan_in, fan_out):
            return np.random.randn(*shape)
            # if self.initialization == "xavier":
            #     return np.random.randn(*shape) * np.sqrt(2 / (fan_in + fan_out))
            # elif self.initialization == "he":
            #     return np.random.randn(*shape) * np.sqrt(2 / fan_in)
            # elif self.initialization == "lecun":
            #     return np.random.randn(*shape) * np.sqrt(1 / fan_in)
            # elif self.initialization == "uniform_xavier":
            #     limit = np.sqrt(6 / (fan_in + fan_out))
            #     return np.random.uniform(-limit, limit, size=shape)
            # elif self.initialization == "uniform_he":
            #     limit = np.sqrt(6 / fan_in)
            #     return np.random.uniform(-limit, limit, size=shape)
            # elif self.initialization == "uniform_lecun":
            #     limit = np.sqrt(3 / fan_in)
            #     return np.random.uniform(-limit, limit, size=shape)
            # else:
            #     raise ValueError("Unsupported initialization method. Choose 'xavier', 'he', 'lecun', 'uniform_xavier', 'uniform_he', or 'uniform_lecun'.")

        # Input to first hidden layer
        self.weights.append(init_weight((hidden_layers[0], input_dim), input_dim, hidden_layers[0]))
        self.biases.append(np.zeros((hidden_layers[0], 1)))

        # Between hidden layers
        for i in range(1, len(hidden_layers)):
            self.weights.append(init_weight((hidden_layers[i], hidden_layers[i-1]), hidden_layers[i-1], hidden_layers[i]))
            self.biases.append(np.zeros((hidden_layers[i], 1)))

        # Last hidden layer to output layer
        self.weights.append(init_weight((output_dim, hidden_layers[-1]), hidden_layers[-1], output_dim))
        self.biases.append(np.zeros((output_dim, 1)))

    def activate(self, x):
        if self.activation == "sigmoid":
            return 1 / (1 + np.exp(-x))
        
    def activation_derivative(self, x):
        if self.activation == "sigmoid":
            sigmoid = self.activate(x)
            return sigmoid * (1 - sigmoid)

    def compute_output(self, input):
        output = self.activate((self.weights[0] @ input) + self.biases[0])
        for i in range(1, len(self.weights)):
            output = self.activate((self.weights[i] @ output) + self.biases[i])
        return output
    
    def cost(self, output, label):
        target = np.zeros((self.output_dim, 1))
        target[label] = 1
        diff_squared = np.square(output - target)
        cost = 0
        for i in range(len(diff_squared)):
            cost += diff_squared[i]
        return cost.item()
    
    def total_cost(self, input_data, input_labels):
        sum = 0
        for i in range(len(input_data)):
            input_layer = self.get_input_layer(input_data[i])
            output_layer = self.compute_output(input_layer)
            sum += self.cost(output_layer, input_labels[i])
        return sum/len(input_data)

    def forward_pass(self, input):
        activations = [input]
        weighted_sums = []

        ws = (self.weights[0] @ input) + self.biases[0]
        act = self.activate(ws)

        weighted_sums.append(ws)
        activations.append(act)

        for i in range(1, len(self.weights)):
            ws = (self.weights[i] @ act) + self.biases[i]
            act = self.activate(ws)

            weighted_sums.append(ws)
            activations.append(act)

        return activations, weighted_sums
        
    
    def compute_gradient(self, input, label):
        activations, weighted_sums = self.forward_pass(input)

        target = np.zeros((self.output_dim, 1))
        target[label] = 1

        bias_derivatives = []
        weight_derivatives = []

        db = self.activation_derivative(weighted_sums[-1]) * 2 * (activations[-1] - target)
        dw = db @ np.transpose(activations[-2])
        da = np.transpose(self.weights[-1]) @ db

        bias_derivatives.insert(0, db)
        weight_derivatives.insert(0, dw)

        for i in range(2, len(self.weights) + 1):
            db = self.activation_derivative(weighted_sums[-i]) * da
            dw = db @ np.transpose(activations[-(i+1)])
            da = np.transpose(self.weights[-i]) @ db

            bias_derivatives.insert(0, db)
            weight_derivatives.insert(0, dw)

        return weight_derivatives, bias_derivatives

    def stochastic_gradient(self, input_data, input_labels, sample_size):
        sample = [random.randint(0, len(input_data) - 1) for _ in range(sample_size)]

        weight_gradient, bias_gradient = self.compute_gradient(
            self.get_input_layer(input_data[sample[0]]),
            input_labels[sample[0]])
        
        for i in range(1, sample_size):
            weight_derivatives, bias_derivatives = self.compute_gradient(
                self.get_input_layer(input_data[sample[i]]),
                input_labels[sample[i]]
            )
            for j in range(len(weight_gradient)):
                weight_gradient[j] = weight_gradient[j] + weight_derivatives[j]
                bias_gradient[j] = bias_gradient[j] + bias_derivatives[j]
            
        for i in range(len(weight_gradient)):
            weight_gradient[i] = weight_gradient[i]/sample_size
            bias_gradient[i] = bias_gradient[i]/sample_size
        
        return weight_gradient, bias_gradient

    def batch_gradient(self, input_data, input_labels):
        weight_gradient, bias_gradient = self.compute_gradient(
            self.get_input_layer(input_data[0]),
            input_labels[0])
        
        for i in range(1, len(input_data)):
            weight_derivatives, bias_derivatives = self.compute_gradient(
                self.get_input_layer(input_data[i]),
                input_labels[i]
            )
            for j in range(len(weight_gradient)):
                weight_gradient[j] = weight_gradient[j] + weight_derivatives[j]
                bias_gradient[j] = bias_gradient[j] + bias_derivatives[j]
            
        for i in range(len(weight_gradient)):
            weight_gradient[i] = weight_gradient[i]/len(input_data)
            bias_gradient[i] = bias_gradient[i]/len(input_data)
        
        return weight_gradient, bias_gradient
    
    def stochastic_gradient_descent(self, input_data, input_labels, sample_size, learning_rate, iterations):
        for _ in range(iterations):
            weight_gradient, bias_gradient = self.stochastic_gradient(input_data, input_labels, sample_size)
            for i in range(len(self.weights)):
                self.weights[i] = self.weights[i] - learning_rate * weight_gradient[i]
                self.biases[i] = self.biases[i] - learning_rate * bias_gradient[i]

    def batch_gradient_descent(self, input_data, input_labels, learning_rate, iterations):
        for _ in range(iterations):
            weight_gradient, bias_gradient = self.batch_gradient(input_data, input_labels)
            for i in range(len(self.weights)):
                self.weights[i] = self.weights[i] - learning_rate * weight_gradient[i]
                self.biases[i] = self.biases[i] - learning_rate * bias_gradient[i]

    def gradient_magnitude(self):
        magnitude = 0
        for i in range(len(self.weights)):
            magnitude += np.inner(self.weights[i].flatten(), self.weights[i].flatten())
            magnitude += np.inner(self.biases[i].flatten(), self.biases[i].flatten())
        return magnitude ** 0.5

    #return model accuracy
    def test(self, test_data, test_labels):
        num_correct = 0
        for i in range(len(test_data)):
            inference = self.make_inference(test_data[i])
            if inference == test_labels[i]:
                num_correct += 1
        return num_correct/len(test_data)
    
    def make_inference(self, input):
        output_layer = self.compute_output(self.get_input_layer(input))
        return np.argmax(output_layer)
    
    def get_input_layer(self, input):
        return np.array(input).flatten().reshape(-1, 1)
    
    def save(self, filename):
        model = [self.weights, self.biases]
        with open(filename, 'wb') as file:
            pickle.dump(model, file)

    def load(self, filename):
        with open(filename, 'rb') as file:
            model = pickle.load(file)
            self.weights = model[0]
            self.biases = model[1]

    def train(self, train_data, train_labels, validate_data, validate_labels, learning_rate, batch_size, max_epochs, patience, filename):
        best_validation_loss = float('inf')
        epochs_without_improvement = 0

        for epoch in range(max_epochs):
            indices = list(range(len(train_data)))
            random.shuffle(indices)
            shuffled_train_data = [train_data[i] for i in indices]
            shuffled_train_labels = [train_labels[i] for i in indices]

            for i in range(0, len(shuffled_train_data), batch_size):
                batch_data = shuffled_train_data[i : i + batch_size]
                batch_labels = shuffled_train_labels[i : i + batch_size]

                weight_gradient, bias_gradient = self.batch_gradient(batch_data, batch_labels)
                for j in range(len(self.weights)):
                    self.weights[j] = self.weights[j] - learning_rate * weight_gradient[j]
                    self.biases[j] = self.biases[j] - learning_rate * bias_gradient[j]

            validation_loss = self.total_cost(validate_data, validate_labels)

            if validation_loss < best_validation_loss:
                best_validation_loss = validation_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print(f"Early stopping at epoch {epoch + 1} with validation loss {validation_loss:.4f}")
                    break

            train_loss = self.total_cost(train_data, train_labels)
            print(f"Epoch {epoch + 1}/{max_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}")

        print("Training complete.")
        self.save(filename)


    def summarize(self):
        """Print a summary of the network's dimensions and parameter shapes."""
        print("Neural Network Summary:")
        print(f"Input dimension: {self.input_dim}")
        print(f"Hidden layers: {self.hidden_layers}")
        print(f"Output dimension: {self.output_dim}")
        print(f"Initialization method: {self.initialization}\n")
        for idx, (w, b) in enumerate(zip(self.weights, self.biases)):
            print(f"Layer {idx + 1} weights shape: {w.shape}")
            print(f"Layer {idx + 1} biases shape: {b.shape}")


In [41]:
nn = NeuralNetwork(input_dim=784, hidden_layers=[16, 16], output_dim=10, initialization="xavier", activation="sigmoid")

In [42]:
nn.total_cost(x_validate, y_validate)

/var/folders/6n/7l19skx97d53w0nbtz56v7rw0000gn/T/ipykernel_65212/1612928832.py:58: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


4.139836505641178

In [32]:
nn.test(x_test, y_test)

/var/folders/6n/7l19skx97d53w0nbtz56v7rw0000gn/T/ipykernel_65212/3011521506.py:57: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


0.8956

In [50]:

nn.train(x_train, y_train, x_validate, y_validate, learning_rate=0.001, batch_size=1000, max_epochs=100, patience=5, filename="model.pickle")

/var/folders/6n/7l19skx97d53w0nbtz56v7rw0000gn/T/ipykernel_65212/1612928832.py:58: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 1/100, Train Loss: 0.2841, Validation Loss: 0.2953
Epoch 2/100, Train Loss: 0.2841, Validation Loss: 0.2953
Epoch 3/100, Train Loss: 0.2841, Validation Loss: 0.2953
Epoch 4/100, Train Loss: 0.2841, Validation Loss: 0.2953
Epoch 5/100, Train Loss: 0.2841, Validation Loss: 0.2953
Epoch 6/100, Train Loss: 0.2841, Validation Loss: 0.2953
Epoch 7/100, Train Loss: 0.2841, Validation Loss: 0.2953
Epoch 8/100, Train Loss: 0.2841, Validation Loss: 0.2953
Epoch 9/100, Train Loss: 0.2841, Validation Loss: 0.2953
Epoch 10/100, Train Loss: 0.2841, Validation Loss: 0.2952
Epoch 11/100, Train Loss: 0.2841, Validation Loss: 0.2952
Epoch 12/100, Train Loss: 0.2841, Validation Loss: 0.2952
Epoch 13/100, Train Loss: 0.2840, Validation Loss: 0.2952
Epoch 14/100, Train Loss: 0.2840, Validation Loss: 0.2952
Epoch 15/100, Train Loss: 0.2840, Validation Loss: 0.2952
Epoch 16/100, Train Loss: 0.2840, Validation Loss: 0.2952
Epoch 17/100, Train Loss: 0.2840, Validation Loss: 0.2952
Epoch 18/100, Train Los